**Important note**: we wrote and ran this notebook on Google Colab for better computing performance The *environment.yaml* and *requirement.txt* files do not provide the adequate libraries for running this notebook. We suggest importing this notebook directly on Colab to run it.

We also used the polars library, an alternative to pandas, for better performance.

# Importing libraries and data

In [ ]:
!pip install umap-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82808 sha256=9b298702ed5bcbd29f313ac9cfe30503befa4e51bbd8971cb567812c286492a9
  Stored in directory: /root/.cache/pip/wheels/a0/e8/c6/a37ea663620bd5200ea1ba0907ab3c217042c1d035ef606acc
  Created wheel for pynndescent: filename=pynndescent-0.5.10-py3-none-any.whl size=55617 sha256=7e841001eafd57f4618930aa092a9cfe4d1c70dac8e4eb38b2165ed1c5376dd2
  Stored in directory: /root/.cache/pip/wheels/4a/38/5d/f60a40a66a9512b7e5e83517ebc2d1b42d857be97d135f1096
Successfully built umap-learn pynndescent


The following line fixes an error encountered while running Umap on a large dataset. See https://github.com/lmcinnes/umap/issues/1008

In [ ]:
pip install --force-reinstall -v "pynndescent==0.5.8"

Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 5.1 MB/s eta 0:00:00
  Running command python setup.py egg_info
  running egg_info
  creating /tmp/pip-pip-egg-info-zpp58gjf/pynndescent.egg-info
  writing /tmp/pip-pip-egg-info-zpp58gjf/pynndescent.egg-info/PKG-INFO
  writing dependency_links to /tmp/pip-pip-egg-info-zpp58gjf/pynndescent.egg-info/dependency_links.txt
  writing requirements to /tmp/pip-pip-egg-info-zpp58gjf/pynndescent.egg-info/requires.txt
  writing top-level names to /tmp/pip-pip-egg-info-zpp58gjf/pynndescent.egg-info/top_level.txt
  writing manifest file '/tmp/pip-pip-egg-info-zpp58gjf/pynndescent.egg-info/SOURCES.txt'
  reading manifest file '/tmp/pip-pip-egg-info-zpp58gjf/pynndescent.egg-info/SOURCES.txt'
  reading manifest template 'MANIFEST.in'
  no previously-included directories found matching 'pynndescent/__pycache__'
  adding license file 'LICENSE'
  writing manifest file '

In [ ]:
import pandas as pd
import polars as pl
import numpy as np
import umap
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfTransformer
import ast


/usr/local/lib/python3.10/dist-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/usr/local/lib/python3.10/dist-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/usr/local/lib/python3.10/dist-packages/umap/distances.py:1086: NumbaDeprecationWarning: The 'nopython' keyword argume

The *tags.csv* file must be imported on Colab.

In [ ]:
df = pl.read_csv(r"tags.csv")

# Processing

We successively apply, for each category of tags:
1. Lowercase every list of tags
2. Remove duplicates and tags appear only once in the dataset
3. One hot encoding each tag
4. tf-idf vectorizer
5. Umap dimensionality reduction (n=2)

In [ ]:
categories = ["subjects", "people","places","times"]

df = pl.read_csv("tags.csv")

df_tags = df.select("work")

for category in categories:
  print(category)

  df_category = (df
  .select(["work",category])
  .drop_nulls()
  )

  df_index_no_nulls = df_category.select("work")

  tfidf_transformer = TfidfTransformer(use_idf=True, smooth_idf=True)
  umap_model = umap.UMAP(n_components=2, random_state=42)

  reduced_data = umap_model.fit_transform(
    tfidf_transformer.fit_transform(
      (df_category
        .with_columns([
            pl.col(category)
              .str.to_lowercase()
              .apply(ast.literal_eval, return_dtype=pl.Object)
              .apply(lambda x: [word for word in set(x) if len(word) >= 3]),
            pl.lit(1).alias("placeholder")
            ])
        .explode(category)
        .pivot(
              index=["work"],
              columns=category,
              values="placeholder",
              sort_columns=False
          )
        .select(pl.exclude("work").fill_null(0))
      )
    )
  )

  df_reduced = pl.concat([
    df_index_no_nulls,
    pl.DataFrame(reduced_data, schema=[category + '_d1', category + '_d2']
  )],
  how = "horizontal"
  )

  df_tags = df_tags.join(df_reduced,on = "work", how = "left").fill_null(0)

subjects


<ipython-input-24-bef27539d7b9>:22: DeprecationWarning: In a future version of polars, the default `aggregate_function` will change from `'first'` to `None`. Please pass `'first'` to keep the current behaviour, or `None` to accept the new one.
  (df_category


people


<ipython-input-24-bef27539d7b9>:22: DeprecationWarning: In a future version of polars, the default `aggregate_function` will change from `'first'` to `None`. Please pass `'first'` to keep the current behaviour, or `None` to accept the new one.
  (df_category
/usr/local/lib/python3.10/dist-packages/umap/spectral.py:260: UserWarning: WARNING: spectral initialisation failed! The eigenvector solver
failed. This is likely due to too small an eigengap. Consider
adding some noise or jitter to your data.

Falling back to random initialisation!
  warn(


places


<ipython-input-24-bef27539d7b9>:22: DeprecationWarning: In a future version of polars, the default `aggregate_function` will change from `'first'` to `None`. Please pass `'first'` to keep the current behaviour, or `None` to accept the new one.
  (df_category
/usr/local/lib/python3.10/dist-packages/umap/spectral.py:260: UserWarning: WARNING: spectral initialisation failed! The eigenvector solver
failed. This is likely due to too small an eigengap. Consider
adding some noise or jitter to your data.

Falling back to random initialisation!
  warn(


times


<ipython-input-24-bef27539d7b9>:22: DeprecationWarning: In a future version of polars, the default `aggregate_function` will change from `'first'` to `None`. Please pass `'first'` to keep the current behaviour, or `None` to accept the new one.
  (df_category


In [ ]:
df_tags.write_csv(r"data\tags_reduced.csv")